# MODEL LOAD

In [1]:
%matplotlib inline

from __future__ import print_function
from PIL import Image
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import torchsummary
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from google.colab import files
from IPython.display import HTML
import time
import shutil
import cv2
import json
import argparse
import os
import numpy as np
import math
import sys
import torchvision.transforms as transforms
from torchvision.utils import save_image
from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable
import torch.nn.functional as F
import torch.autograd as autograd
from torch import squeeze
from torch import Tensor
from IPython.display import clear_output

manualSeed = 999

random.seed(manualSeed)
torch.manual_seed(manualSeed)

In [2]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/MyDrive')

Mounted at /content/MyDrive


In [3]:
workers = 2
batch_size = 128
image_size = 512
nc = 1
nz = 100
ngf = 64
ndf = 64
num_epochs = 5
lr = 0.0002
beta1 = 0.5
ngpu = 1

# For new model setup
os.makedirs("images", exist_ok=True)

n_epochs = 200
#batch_size=100
lr=0.0002
b1=0.5
b2=0.999
n_cpu=8
latent_dim=100
img_size=image_size
channels=1
n_critic=5 # help="number of training steps for discriminator per iter")
clip_value=0.01 # help="lower and upper clip value for disc. weights")
sample_interval=400 # help="interval betwen image samples")

img_shape = (channels, img_size, img_size)

device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

In [4]:
# custom weights initialization called on netG and netD
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

### GENERATOR

In [5]:
# Generator Code

# Spatial size of training images. All images will be resized to this
#   size using a transformer.
nc = 1
image_size = 512
ngf = int(64/4)

# Generator Code

# Spatial size of training images. All images will be resized to this
#   size using a transformer.

class Generator(nn.Module):
    def __init__(self, ngpu):
        super(Generator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # input is Z, going into a convolution
            nn.ConvTranspose2d( nz, ngf * 64, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 64),
            nn.ReLU(True),
            # state size. (ngf*16) x 4 x 4
            nn.ConvTranspose2d(ngf * 64, ngf * 32, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 32),
            nn.ReLU(True),
            # state size. (ngf*16) x 4 x 4
            nn.ConvTranspose2d(ngf * 32, ngf * 16, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 16),
            nn.ReLU(True),
            # state size. (ngf*16) x 4 x 4
            nn.ConvTranspose2d(ngf * 16, ngf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            # state size. (ngf*8) x 8 x 8
            nn.ConvTranspose2d( ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # state size. (ngf*4) x 16 x 16
            nn.ConvTranspose2d( ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # state size. (ngf*2) x 32 x 32
            nn.ConvTranspose2d( ngf * 2,     ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            # state size. (ngf) x 64 x 64
            nn.ConvTranspose2d( ngf,        nc, 4, 2, 1, bias=False),
            nn.Tanh()
            # state size. (nc) x 128 x 128
        )

    def forward(self, input):
        return self.main(input)

# Create the generator
generator = Generator(ngpu).to(device)


# Handle multi-gpu if desired
if (device.type == 'cuda') and (ngpu > 1):
    generator = nn.DataParallel(generator, list(range(ngpu)))

# Apply the weights_init function to randomly initialize all weights
#  to mean=0, stdev=0.02.
generator.apply(weights_init)

torchsummary.summary(generator,input_size=(100,1,1))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
   ConvTranspose2d-1           [-1, 1024, 4, 4]       1,638,400
       BatchNorm2d-2           [-1, 1024, 4, 4]           2,048
              ReLU-3           [-1, 1024, 4, 4]               0
   ConvTranspose2d-4            [-1, 512, 8, 8]       8,388,608
       BatchNorm2d-5            [-1, 512, 8, 8]           1,024
              ReLU-6            [-1, 512, 8, 8]               0
   ConvTranspose2d-7          [-1, 256, 16, 16]       2,097,152
       BatchNorm2d-8          [-1, 256, 16, 16]             512
              ReLU-9          [-1, 256, 16, 16]               0
  ConvTranspose2d-10          [-1, 128, 32, 32]         524,288
      BatchNorm2d-11          [-1, 128, 32, 32]             256
             ReLU-12          [-1, 128, 32, 32]               0
  ConvTranspose2d-13           [-1, 64, 64, 64]         131,072
      BatchNorm2d-14           [-1, 64,

### DISCRIMINATOR / CRITIC

In [6]:
# Discriminator Code

# Spatial size of training images. All images will be resized to this
#   size using a transformer.
image_size = 512
ndf = int(64/4)

class Discriminator(nn.Module):
    def __init__(self, ngpu):
        super(Discriminator, self).__init__()
        self.ngpu = ngpu

        self.feature_ext = nn.Sequential(
            # input is (nc) x 128 x 128
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf) x 64 x 64

            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            #nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*2) x 32 x 32

            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            #nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*4) x 16 x 16
            
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            #nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*8) x 8 x 8

            nn.Conv2d(ndf * 8, ndf * 16, 4, 2, 1, bias=False),
            #nn.BatchNorm2d(ndf * 16),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(ndf * 16, ndf * 32, 4, 2, 1, bias=False),
            #nn.BatchNorm2d(ndf * 16),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(ndf * 32, ndf * 64, 4, 2, 1, bias=False),
            #nn.BatchNorm2d(ndf * 16),
            nn.LeakyReLU(0.2, inplace=True),
        )

        self.feature_agg = nn.Sequential(
           # state size. (ndf*16) x 4 x 4
            nn.Conv2d(ndf * 64, 1, 4, 1, 0, bias=False),
            
            ################ SIgmoid fjernet
            #nn.Sigmoid()
            # state size. 1 x 1 x 1
        )

    def forward(self, input):
        feature = self.feature_ext(input)
        out = self.feature_agg(feature)

        out = squeeze(out,3)
        out = squeeze(out,2)
        return out

    def feature_forward(self, input):
        feature = self.feature_ext(input)

        return feature

# Create the Discriminator
discriminator = Discriminator(ngpu).to(device)

# Handle multi-gpu if desired
if (device.type == 'cuda') and (ngpu > 1):
    discriminator = nn.DataParallel(discriminator, list(range(ngpu)))
    
# Apply the weights_init function to randomly initialize all weights
#  to mean=0, stdev=0.2.
discriminator.apply(weights_init)

torchsummary.summary(discriminator,input_size=(nc,image_size,image_size))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 16, 256, 256]             256
         LeakyReLU-2         [-1, 16, 256, 256]               0
            Conv2d-3         [-1, 32, 128, 128]           8,192
         LeakyReLU-4         [-1, 32, 128, 128]               0
            Conv2d-5           [-1, 64, 64, 64]          32,768
         LeakyReLU-6           [-1, 64, 64, 64]               0
            Conv2d-7          [-1, 128, 32, 32]         131,072
         LeakyReLU-8          [-1, 128, 32, 32]               0
            Conv2d-9          [-1, 256, 16, 16]         524,288
        LeakyReLU-10          [-1, 256, 16, 16]               0
           Conv2d-11            [-1, 512, 8, 8]       2,097,152
        LeakyReLU-12            [-1, 512, 8, 8]               0
           Conv2d-13           [-1, 1024, 4, 4]       8,388,608
        LeakyReLU-14           [-1, 102

### LOAD STATE

In [7]:
n_epochs = 200
filename = '512_WGGAN_lessdeep_05MeanSTD'

# Load Generator weights from Google Drive
state_dict_G = torch.load('/content/MyDrive/MyDrive/0-MasterThesis/GAN-TESTS_JUNE/' + filename + '/generator_' + filename + '_epch_200' + '.pth')
state_dict_D = torch.load('/content/MyDrive/MyDrive/0-MasterThesis/GAN-TESTS_JUNE/' + filename + '/discriminator_' + filename + '_epch_200' + '.pth')
#workspace =  torch.load('/content/MyDrive/MyDrive/0-MasterThesis/GAN-TESTS_JUNE/workspace_' + filename + '.json')
workspace  =  torch.load('/content/MyDrive/MyDrive/0-MasterThesis/GAN-TESTS_JUNE/' + filename + '/workspace_' + filename + '_epch_200' + '.json')

# Load Discriminator weights from Google Drive
generator.load_state_dict(state_dict_G)
discriminator.load_state_dict(state_dict_D)
g_loss_log = workspace['g_loss_log']
d_loss_log = workspace['d_loss_log']
iters = workspace['iters']
iters_epch = workspace['iters_epch']

epoch_offset = iters_epch

# Actual Evaluation

In [8]:
%matplotlib inline
import os
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import numpy as np
from PIL import Image
import shutil
import pandas as pd
import cv2
from IPython.display import clear_output
import os
from sklearn import preprocessing
import matplotlib.patches as patches
import time
from termcolor import colored

In [9]:
%mkdir -p /content/Eval_Data
!cp /content/MyDrive/MyDrive/0-MasterThesis/Eval_Data.zip /content/Eval_Data
%cd /content/Eval_Data/
!unzip Eval_Data.zip
%cd /content/

Streaming output truncated to the last 5000 lines.
  inflating: Haak_data/Flight3/20170926_norm_1100-3900/01895.png  
  inflating: Haak_data/Flight3/20170926_norm_1100-3900/01896.png  
  inflating: Haak_data/Flight3/20170926_norm_1100-3900/01897.png  
  inflating: Haak_data/Flight3/20170926_norm_1100-3900/01898.png  
  inflating: Haak_data/Flight3/20170926_norm_1100-3900/01899.png  
  inflating: Haak_data/Flight3/20170926_norm_1100-3900/01900.png  
  inflating: Haak_data/Flight3/20170926_norm_1100-3900/01901.png  
  inflating: Haak_data/Flight3/20170926_norm_1100-3900/01902.png  
  inflating: Haak_data/Flight3/20170926_norm_1100-3900/01903.png  
  inflating: Haak_data/Flight3/20170926_norm_1100-3900/01904.png  
  inflating: Haak_data/Flight3/20170926_norm_1100-3900/01905.png  
  inflating: Haak_data/Flight3/20170926_norm_1100-3900/01906.png  
  inflating: Haak_data/Flight3/20170926_norm_1100-3900/01907.png  
  inflating: Haak_data/Flight3/20170926_norm_1100-3900/01908.png  
  inflating

In [10]:
Haak1_true_df = pd.read_csv(r'/content/Eval_Data/Haak_data/Flight1/Haak1_true_annot.csv')
Haak1_anomalies_object_df = pd.read_csv(r'/content/Eval_Data/Haak_data/Flight1/Haak1_ano_object_annot.csv')
Haak1_anomalies_land_df = pd.read_csv(r'/content/Eval_Data//Haak_data/Flight1/Haak1_ano_land_annot.csv')

Haak3_true_df = pd.read_csv(r'/content/Eval_Data//Haak_data/Flight3/Haak3_true_annot.csv')
Haak3_anomalies_object_df = pd.read_csv(r'/content/Eval_Data//Haak_data/Flight3/Haak3_ano_object_annot.csv')
Haak3_anomalies_land_df = pd.read_csv(r'/content/Eval_Data//Haak_data/Flight3/Haak3_ano_land_annot.csv')

Danilo_true_df = pd.read_csv(r'/content/Eval_Data//Danilo_data/IR/Danilo_true_annot.csv')
Danilo_anomalies_land_df = pd.read_csv(r'/content/Eval_Data/Danilo_data/IR/Danilo_ano_land_annot.csv')
Empty_df = pd.read_csv(r'/content/Eval_Data/Danilo_data/IR/Empty.csv')

# Haak - Flight 1
Haak1_true_train       = Haak1_true_df.sample(frac = 0.7, random_state = 100)
Haak1_true_test        = Haak1_true_df.drop(Haak1_true_train.index)

Haak1_ano_object_train = Haak1_anomalies_object_df.sample(frac = 0.7, random_state = 100)
Haak1_ano_object_test  = Haak1_anomalies_object_df.drop(Haak1_ano_object_train.index)
Haak1_ano_object       = Haak1_anomalies_object_df

Haak1_ano_land_train   = Haak1_anomalies_land_df.sample(frac = 0.7, random_state = 100)
Haak1_ano_land_test    = Haak1_anomalies_land_df.drop(Haak1_ano_land_train.index)
Haak1_ano_land         = Haak1_anomalies_land_df
 
# Haak - Flight 1
Haak3_true             = Haak3_true_df
Haak3_ano_object       = Haak3_anomalies_object_df
Haak3_ano_land         = Haak3_anomalies_land_df

# Danilo
Danilo_true            = Danilo_true_df
Danilo_ano_land        = Danilo_anomalies_land_df
empty                  = Empty_df

'''
Haak1_true_train | Haak1_true_test
Haak1_ano_object
Haak1_ano_land

Haak3_true 
Haak3_ano_object 
Haak3_ano_land 

Danilo_true
empty
Danilo_ano_land
'''

'\nHaak1_true_train | Haak1_true_test\nHaak1_ano_object\nHaak1_ano_land\n\nHaak3_true \nHaak3_ano_object \nHaak3_ano_land \n\nDanilo_true\nempty\nDanilo_ano_land\n'

# Eval

In [ ]:
df = Haak1_true_train
directory = '/content/Eval_Data/Haak_data/Flight1/20170926_norm_1100-3900/' # 8-bit
scenery = 'Haak1 - true'
isAnomObject = False
isAnomLand   = False
isTrue       = True

resize = False

##################################################### HAAK 1
#####################################################
df = Haak1_true_test
directory = '/content/Eval_Data/Haak_data/Flight1/20170926_norm_1100-3900/' # 8-bit
scenery = 'Haak1 - true'
isAnomObject = False
isAnomLand   = False
isTrue       = True

resize = False
#####################################################
df = Haak1_ano_object
directory = '/content/Eval_Data/Haak_data/Flight1/20170926_norm_1100-3900/' # 8-bit
scenery = 'Haak1 - object'
isAnomObject = True
isAnomLand   = False
isTrue       = False

resize = False
#####################################################
df = Haak1_ano_land
directory = '/content/Eval_Data/Haak_data/Flight1/20170926_norm_1100-3900/' # 8-bit
scenery = 'Haak1 - land'
isAnomObject = False
isAnomLand   = True
isTrue       = False

resize = False
##################################################### HAAK 3
#####################################################
df = Haak3_true
directory = '/content/Eval_Data/Haak_data/Flight3/20170926_norm_1100-3900/' # 8-bit
scenery = 'Haak3 - true'
isAnomObject = False
isAnomLand   = False
isTrue       = True

resize = False
#####################################################
df = Haak3_ano_object
directory = '/content/Eval_Data/Haak_data/Flight3/20170926_norm_1100-3900/' # 8-bit
scenery = 'Haak3 - object'
isAnomObject = True
isAnomLand   = False
isTrue       = False

resize = False
#####################################################
df = Haak3_ano_land
directory = '/content/Eval_Data/Haak_data/Flight3/20170926_norm_1100-3900/' # 8-bit
scenery = 'Haak3 - land'
isAnomObject = False
isAnomLand   = True
isTrue       = False

resize = False
##################################################### DANILO
#####################################################
df = Danilo_true
directory = '/content/Eval_Data/Danilo_data/IR/All-Frames-Simple/' # 8-bit
scenery = 'Danilo - true'
isAnomObject = False
isAnomLand   = False
isTrue       = True

resize = True
#####################################################
df = Danilo_ano_land
directory = '/content/Eval_Data/Danilo_data/IR/All-Frames-Simple/' # 8-bit
scenery = 'Danilo - land'
isAnomObject = False
isAnomLand   = True
isTrue       = False

resize = True

In [15]:
df = Haak3_true
directory = '/content/Eval_Data/Haak_data/Flight3/20170926_norm_1100-3900/' # 8-bit
scenery = 'Haak3 - true'
isAnomObject = False
isAnomLand   = False
isTrue       = True

resize = False
#####################################################
area_threshold = 20

doPlots = False
#doExhaustivePlots = False # DOESNT DIFFER FROM NORMAL PLOT
doErrorValidation = False
doNMSValidation = False
#####################################################
n_files = len(df)
start = 0
end = n_files
#end = 50

metrics = {'FPs':0, 'FNs':0, 'TPs': 0}
residual_all_img = []
loss_all_img = []
j = 0
#####################################################
for i, row in df[start:end].iterrows():
    j += 1
    ############################
    true_boxes = []
    pred_boxes = []
    intersection_boxes = []
    FP, FN, TP = 0,0,0
    ############################
    
    print(str(j) + '/' + str(n_files))
    file = row['file']
    
    #img = np.array(Image.open(os.path.join(directory, file)))
    #img = cv2.imread(os.path.join(directory, file))
    img = Image.open(os.path.join(directory, file))

    if resize:
      img = cv2.imread(os.path.join(directory, file))
      img = cv2.resize(img, (640,512), interpolation = cv2.INTER_CUBIC)
      img = Image.fromarray(img)



    ##################### ALGORITHM #####################
    start_time = time.time()

    t = transforms.Compose([
                transforms.Grayscale(num_output_channels=1),
                transforms.Resize(image_size),
                transforms.CenterCrop(image_size),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.5], std=[0.5]),
                ])
    

    objective_img_t = t(img)

    fake_img, objective_img, residual_img, loss_list, res_loss_list, disc_loss_list = AnoGAN_inference(objective_img_t, doPlots)
    print('final loss: ', str(loss_list[-1]))

    residual_all_img = [*residual_all_img, residual_img]
    loss_all_img = [*loss_all_img, loss_list[-1]]



    #time.sleep(5)

    print("--- %s seconds ---" % (time.time() - start_time))

    if (i==0) or (i % 500 == 0) or (i == (end-1)):
      Haak3_true_res_  = np.array(residual_all_img)
      Haak3_true_loss_ = np.array(loss_all_img)

      np.save('/content/Haak3_true_res_{}'.format(j), Haak3_true_res_)
      np.save('/content/Haak3_true_loss_{}'.format(j), Haak3_true_loss_)

      zip_file = '/content/Haak3_true_res_{}'.format(j)

      print(zip_file)

      !zip -r {zip_file}.zip {zip_file}.npy




Streaming output truncated to the last 5000 lines.
tensor(27072.5664)
final loss:  27070.2265625
--- 9.651106834411621 seconds ---
4452/4835
tensor(27014.6191)
tensor(26951.5000)
tensor(26890.4375)
tensor(26828.2129)
tensor(26740.1543)
tensor(26747.5625)
tensor(26743.1953)
tensor(26740.7793)
tensor(26735.9473)
tensor(26733.1602)
final loss:  26728.93359375
--- 9.691540718078613 seconds ---
4453/4835
tensor(27094.2598)
tensor(26646.9414)
tensor(26684.7891)
tensor(26679.4414)
tensor(26657.8203)
tensor(26636.4004)
tensor(26588.9590)
tensor(26595.3320)
tensor(26600.4746)
tensor(26600.2266)
final loss:  26593.701171875
--- 9.666833877563477 seconds ---
4454/4835
tensor(27073.8496)
tensor(26678.9199)
tensor(26679.5586)
tensor(26677.4336)
tensor(26671.0195)
tensor(26667.8730)
tensor(26663.5879)
tensor(26662.1836)
tensor(26659.0059)
tensor(26657.2285)
final loss:  26654.5546875
--- 9.656166315078735 seconds ---
4455/4835
tensor(27274.8184)
tensor(27324.8789)
tensor(27233.6797)
tensor(27122.523

In [16]:
while True:
  print('Hey')
  time.sleep(600)

Hey
Hey


KeyboardInterrupt: ignored

In [17]:
!cp /content/Haak3_true_res_4835.zip /content/MyDrive/MyDrive/0-MasterThesis/

# RESIDUAL EVALUATION

In [36]:
time.sleep(100)

from google.colab import files
files.download('/content/Haak3_true_res_4835.npy')
files.download('/content/Haak3_true_res_4835.zip')
files.download('/content/Haak3_true_loss_4835.zip') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

FileNotFoundError: ignored

In [ ]:
#Haak1_obj_res  = np.array(residual_all_img)
#Haak1_obj_loss = np.array(loss_all_img)
#Haak1_true_res  = np.array(residual_all_img)
#Haak1_true_loss = np.array(loss_all_img)
#Haak1_land_res  = np.array(residual_all_img)
#Haak1_land_loss = np.array(loss_all_img)
Haak3_obj_loss = np.array(loss_all_img)
Haak3_obj_res = np.array(loss_all_img)

In [22]:
np.save('/content/Haak3_obj_loss', Haak3_obj_loss)
np.save('/content/Haak3_obj_res', Haak3_obj_res)

In [27]:
!zip -r /content/Haak1_true_res_vf.zip /content/Haak1_true_res_vf.npy 

  adding: content/Haak1_true_res_vf.npy (deflated 15%)


In [26]:
!cp /content/Haak1_true_res_vf.zip /content/MyDrive/MyDrive/0-MasterThesis/

In [ ]:
df = Haak1_ano_object
directory = '/content/Eval_Data/Haak_data/Flight1/20170926_norm_1100-3900/' # 8-bit
scenery = 'Haak1 - object'
isAnomObject = True
isAnomLand   = False
isTrue       = False
residual_list = Haak1_obj_res
loss_lis = Haak1_obj_loss
#####################################################
area_threshold = 20

doPlots = False
#doExhaustivePlots = False # DOESNT DIFFER FROM NORMAL PLOT
doErrorValidation = False
doNMSValidation = False
#####################################################
n_files = len(df)
start = 0
end = n_files
#end = 50

metrics = {'FPs':0, 'FNs':0, 'TPs': 0}
residual_all_img = []
loss_all_img = []
j = 0
#####################################################
for i, row in df[start:end].iterrows():
    j += 1
    ############################
    true_boxes = []
    pred_boxes = []
    intersection_boxes = []
    FP, FN, TP = 0,0,0
    ############################
    
    print(str(j) + '/' + str(n_files))
    file = row['file']
    img = Image.open(os.path.join(directory, file))

    ##################### ALGORITHM #####################
    start_time = time.time()

    residual = residual_list[i]

    mask = cv2.threshold(residual, 0.05,10,cv2.THRESH_BINARY)
    
    #time.sleep(5)

    print("--- %s seconds ---" % (time.time() - start_time))
    #####################################################

    if isAnomLand or isAnomObject:
        true_boxes = [row[['x','y','w','h']].to_numpy()]

    # Pred Boxes
    #pred_boxes = area_thresh(pred_boxes, area_threshold)
    pred_boxes = boxes_from_mask(mask, area_threshold)
    
    if (not isAnomLand) and (len(pred_boxes) >= 2):
        if doNMSValidation: 
            plot_results(img, pred_boxes, true_boxes, intersection_boxes, 'N/A')
            print(colored('NON MAXIMAL SUPRESSION', 'magenta'))
            print('Prior: {}'.format(pred_boxes))
            pred_boxes = pred_box_IoU_cancellation(pred_boxes)
            print('Post: {}'.format(pred_boxes))
            _ = input()
        else:
            pred_boxes = pred_box_IoU_cancellation(pred_boxes)
    
    pre_land_boxes = []

    if isAnomLand:
        pre_land_boxes = pred_boxes
        pred_boxes = covering_rectangle_from_square_boxes(pred_boxes) # Combine small boxes

    # Calculate accuracy
    intersection_boxes, FP, FN, TP = compare_predictions_with_truth(pred_boxes, true_boxes)


    # Metrics
    update = {'FPs':FP, 'FNs':FN, 'TPs': TP}
    metrics = {i: metrics.get(i, 0) + update.get(i, 0) for i in set(update)}

    # Plot
    title = 'FP: {}, FN: {}, TP: {} | {}'.format(FP, FN, TP, scenery)

    #if doPlots:
    #    plot_results(img, pred_boxes, true_boxes, intersection_boxes, title)
        
    #if doExhaustivePlots:
    #    exhaustive_plot_results(img, img_canny, img_dil, mask, pred_boxes, true_boxes, intersection_boxes, pre_land_boxes, title)

    if doErrorValidation:
        if FP:
            plot_results(img, pred_boxes, true_boxes, intersection_boxes, title)
            print(colored('FALSE POSITIVE, total: {}'.format(metrics['FPs']), 'cyan'))
            print(row)
            print(pred_boxes)
            _ = input()
        if FN:
            plot_results(img, pred_boxes, true_boxes, intersection_boxes, title)
            print(colored('FALSE NEGATIVE, total: {}'.format(metrics['FNs']), 'red'))
            print(row)
            _ = input()

   

print('\n\n######## frames: {} mask threshold: {} | gaussian threshold: {} ########'.format(n_files, threshold_mask, gaussian_mask))
print('Object: {}'.format(metrics))
print('Recall: {:.4f}'.format(metrics['TPs']/(metrics['FNs']+metrics['TPs']+0.00000001)))
print('Precision: {:.4f}'.format(metrics['TPs']/(metrics['FPs']+metrics['TPs']+0.00000001)))



#        Flight 1    Flight 2      Flight 3
#   Rec
#   Preec

(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)

In [11]:
from torch.autograd import Variable
from IPython.display import clear_output

def AnoGAN_inference(objective_img_t, doPlots):
    i = 0
    Lambda= 0.1
    lr = 0.01
    noise = Variable(torch.randn(1, nz, 1, 1, device=device), requires_grad=True)
    noise_optimizer = optim.Adam([noise], lr=lr)

    loss_list = []
    res_loss_list = []
    disc_loss_list = []

    for i in range(1000):

        if (i%200==0)and (i != 0):
          lr = lr / 2
          noise_optimizer = optim.Adam([noise], lr=lr)

          

        fake_img_t = generator(noise)
        loss, res_loss, disc_loss = anomaly_score(Variable(objective_img_t).cuda(), fake_img_t, Lambda=Lambda)

        loss.backward()
        noise_optimizer.step()

        loss_list.append(float(loss.clone().detach().cpu().numpy()))
        res_loss_list.append(float(res_loss.clone().detach().cpu().numpy()))
        disc_loss_list.append(float(disc_loss.clone().detach().cpu().numpy()))
        
        if i%100==0:
          print(loss.cpu().data)
          #print(i)
          
          if doPlots:
            fake_img_t = generator(noise)

            fake_img_t_copy = fake_img_t
            fake_img = fake_img_t_copy.to(device)[:1].cpu().detach().numpy()[0,0,:,:]

            objective_img_copy = objective_img_t
            objective_img = objective_img_copy.cpu().detach().numpy()[0,:,:]

            residual_img = objective_img - fake_img


            plot_progress(Lambda, objective_img, fake_img, noise, loss_list, res_loss_list, disc_loss_list)


    fake_img_t = generator(noise)
    fake_img = fake_img_t.to(device)[:1].cpu().detach().numpy()[0,0,:,:]
    objective_img = objective_img_t.cpu().detach().numpy()[0,:,:]
    residual_img = objective_img - fake_img

    return fake_img, objective_img, residual_img, loss_list, res_loss_list, disc_loss_list

In [12]:
def anomaly_score(x,G_z,Lambda=0.1):
    x_feature = discriminator.feature_forward(x)
    G_z_feature = discriminator.feature_forward(G_z)
    
    residual_loss = torch.sum(torch.abs(G_z-x))
    #discrimination_loss = torch.sum(torch.abs(x_feature-G_z_feature))
    discrimination_loss = torch.sum(torch.abs(x_feature-G_z_feature))

    #median_loss = torch.median(torch.abs(x_feature-G_z_feature))
    
    total_loss = (1-Lambda)*residual_loss + Lambda*discrimination_loss
    return total_loss, residual_loss, discrimination_loss

In [13]:
def plot_batch(cracked_batch, noise, batch_size, i):
    fake_batch = generator(noise)

    created_img = fake_batch.to(device)[:1].cpu().detach().numpy()[0,0,:,:]
    objective_img = cracked_batch[0].cpu().detach().numpy()[0,:,:]

    min_ = objective_img.min()
    max_ = objective_img.max()
    print('min: {:.2f}'.format(min_))
    print('max: {:.2f}'.format(max_))
    
    #min_ = -0.5
    #max_ = 0.5

    residual = objective_img - created_img

    plt.figure(figsize=(21, 21))
    plt.subplot(2,2,1)
    plt.imshow(objective_img, cmap='gray', vmin = min_, vmax = max_)
    plt.colorbar()

    plt.subplot(2,2,2)
    plt.imshow(created_img, cmap='gray', vmin = min_, vmax = max_)
    plt.colorbar()

    plt.subplot(2,2,3)
    plt.imshow(residual, cmap='gray', vmin = min_, vmax = max_)
    plt.colorbar()

    residual_offset = residual
    plt.subplot(2,2,4)
    residual_thrsh = cv2.threshold(residual_offset, 0.05,10,cv2.THRESH_BINARY)
    plt.imshow(residual_thrsh[1], cmap='gray')

    plt.show()

In [14]:
def plot_progress(Lambda, objective_img, fake_img, noise, loss_list, res_loss_list, disc_loss_list):

  plt.figure(figsize=(34,14))

  ## Loss
  plt.subplot(2,3,1)

  plt.plot(loss_list, label='Loss', linewidth=3.5)
  plt.plot(res_loss_list, label='Res loss {:.2f}'.format(1-Lambda))
  plt.plot(disc_loss_list, label='Disc loss {:.2f}'.format(Lambda))
  plt.legend()
  plt.title('loss')
  plt.grid()
  plt.ylim([0, max([*loss_list, *res_loss_list, *disc_loss_list]) + 250])
  plt.xlabel('iterations') 
  plt.ylabel('')

  #######
  plt.subplot(2,3,4)
  plt.hist(fake_img.flatten(), bins=30, density=True,
          range=(-0.3,0.), alpha=0.4)
  plt.hist(objective_img.flatten(), bins=30, color='r', density=True,
          range=(-0.3,0.), alpha=0.4)

  #######

  min_ = objective_img.min()
  max_ = objective_img.max()
  #print('min: {:.2f}'.format(min_))
  #print('max: {:.2f}'.format(max_))
  
  #min_ = -0.5
  #max_ = 0.5

  residual = np.abs(objective_img - fake_img)

  plt.subplot(2,3,2)
  plt.imshow(objective_img, cmap='gray', vmin = min_, vmax = max_)
  plt.colorbar()

  plt.subplot(2,3,3)
  plt.imshow(fake_img, cmap='gray', vmin = min_, vmax = max_)
  plt.colorbar()

  plt.subplot(2,3,5)
  plt.imshow(residual, cmap='gray')
  plt.colorbar()

  plt.subplot(2,3,6)
  residual_thrsh = cv2.threshold(residual, 0.05,10,cv2.THRESH_BINARY)[1]
  #residual_thrsh = cv2.threshold(np.array(residual, dtype=np.uint8), 0, 0.2, cv2.THRESH_BINARY+cv2.THRESH_OTSU)[1]
  plt.imshow(residual_thrsh, cmap='gray')


  plt.show()


def pred_box_IoU_cancellation(pred_boxes):
    
    boxes_to_save = np.ones(len(pred_boxes))
    for i, box1 in enumerate(pred_boxes):       
        for j, box2 in enumerate(pred_boxes):
            if i == j:
                continue

            _, _, _, _, A_intsc, A_union, IoU, A1, A2  = intersection_info(box1, box2)
            
            # if two predicted boxes are almost equal (50% of area in another, remove the smallest)
            if (A_intsc > 0.25*A1) or (A_intsc > 0.25*A2):
                if A1 >= A2:
                    boxes_to_save[j] = 0
                else:
                    boxes_to_save[i] = 0
                  
    boxes = []
    for k, boolean in enumerate(boxes_to_save):
        if boolean:
            box = pred_boxes[k]
            boxes = [*boxes, box]
    boxes = np.array(boxes)
    
    return boxes


def covering_rectangle_from_square_boxes(boxes):
    output = np.array([])
    
    if len(boxes):
        x_left = np.min(boxes[:,0])
        width = np.max(boxes[:,0] + boxes[:,2] - x_left)
        y_top = np.min(boxes[:,1])
        height = np.max(boxes[:,1] + boxes[:,3] - y_top)
        
        output = np.array([[x_left, y_top, width, height]])

    return output

def intersection_info(box_i, box_j):
    # NMS
    x_left_i, y_upper_i, w_i, h_i = box_i
    x_right_i = x_left_i + w_i
    y_lower_i = y_upper_i + h_i
    area_i = w_i*h_i

    x_left_j, y_upper_j, w_j, h_j = box_j
    x_right_j = x_left_j + w_j
    y_lower_j = y_upper_j + h_j
    area_j = w_j*h_j

    x_left  = max(x_left_i, x_left_j)
    y_upper = max(y_upper_i, y_upper_j)

    x_right = min(x_right_i, x_right_j)
    y_lower = min(y_lower_i, y_lower_j)

    w_intersec = max(0, x_right - x_left)
    h_intersec = max(0, y_lower - y_upper)

    intersection_area = w_intersec*h_intersec
    union_area = area_i + area_j - intersection_area
    IoU = intersection_area / union_area
    
    return list([x_left, y_upper, w_intersec, h_intersec, intersection_area, union_area, IoU, area_i, area_j])

def compare_predictions_with_truth(pred_boxes, true_boxes):
    
    false_positives = 0
    false_negatives = 0
    true_positives = 0
    
    
    intersection_boxes = []
    match_found_pred = np.ones(len(pred_boxes))
    match_found_true = np.ones(len(true_boxes))
    
    for i, box_pred in enumerate(pred_boxes):
        for j, box_true in enumerate(true_boxes):
                     
            x_left, y_upper, w_intsc, h_intsc, A_intsc, A_union, IoU, A_pred, A_true  = intersection_info(box_pred, box_true)

            # If IoU big enough
            if IoU >= 0.1:
                match_found_pred[i] = 0
                match_found_true[j] = 0
                intersection_boxes = [*intersection_boxes, [x_left, y_upper, w_intsc, h_intsc]]
            
            # If within on another, and area large enough
            elif (IoU >= 0.05) and ((A_intsc>=A_pred) or (A_intsc>=A_true)):
                match_found_pred[i] = 0
                match_found_true[j] = 0
                intersection_boxes = [*intersection_boxes, [x_left, y_upper, w_intsc, h_intsc]]
                
            # If within on another, but area small, only false neg, not false positve
            elif (IoU < 0.05) and ((A_intsc>=A_pred) or (A_intsc>=A_true)):
                match_found_pred[i] = 0
            
    intersection_boxes = np.array(intersection_boxes)
        
    false_positives = match_found_pred.sum()
    false_negatives = match_found_true.sum()
    true_positives  = len(match_found_true) - false_negatives
                
    return intersection_boxes, false_positives, false_negatives, true_positives

'''
def area_thresh(pred_boxes, area_threshold):
    
    # Threshold by area
    contours_areathrsh = []
    for cnt in pred_boxes:
        cnt = np.array(cnt).astype(np.uint8)
        print(cnt)
        if cv2.contourArea(cnt) > area_threshold:
            contours_areathrsh = [*contours_areathrsh, cnt]
    contours_areathrsh = np.array(contours_areathrsh)

    return contours_areathrsh
'''

def boxes_from_mask(mask, area_threshold):
    
    contours, hierarchy = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, 2)
    
    # Threshold by area
    contours_areathrsh = []
    for cnt in contours:
        if cv2.contourArea(cnt) > area_threshold:
            contours_areathrsh = [*contours_areathrsh, cnt]
    contours_areathrsh = np.array(contours_areathrsh)
    
    # combine boxes thresholded
    boxes = []
    for i, cnt in enumerate(contours_areathrsh):
        box = cv2.boundingRect(cnt)
        boxes = [*boxes, box]
    boxes = np.array(boxes)

    return boxes

def plot_results(img, pred_boxes, true_boxes, intersection_boxes, title):
    colors = ['r', 'c', 'g', 'b', 'y', 'm']
    
    clear_output(wait=True)
    fig = plt.figure(figsize=(21,10))

    # Plot boxes - Area thresholded
    plt.subplot(1,2,1)
    plt.imshow(img, cmap='gray', vmin=0, vmax=255)
    ax = plt.gca()
    
    plt.subplot(1,2,2)
    plt.imshow(img, cmap='gray', vmin=0, vmax=255)
    ax = plt.gca()

    for i, box in enumerate(intersection_boxes):
        x,y,w,h = box # upper left
        rect = patches.Rectangle((x,y), w,h, edgecolor='y', facecolor='y', fill=True, linewidth=2, alpha=0.25)
        ax.add_patch(rect)     
    
    for i, box in enumerate(pred_boxes):
        x,y,w,h = box # upper left
        rect = patches.Rectangle((x,y), w,h, edgecolor='r', fill=False, linewidth=2)
        ax.add_patch(rect)
        
    for i, box in enumerate(true_boxes):
        x,y,w,h = box # upper left
        rect = patches.Rectangle((x,y), w,h, edgecolor='g', fill=False, linewidth=2)
        ax.add_patch(rect)           
 
    fig.suptitle(title + ' | Found: {}, Truth: {}'.format(len(pred_boxes), len(true_boxes)), fontsize=18)
    
    plt.show()
    
    
def exhaustive_plot_results(img, img_canny, img_dil, mask, pred_boxes, true_boxes, intersection_boxes, pre_land_boxes, title):
    colors = ['r', 'c', 'g', 'b', 'y', 'm']
    
    clear_output(wait=True)
    fig = plt.figure(figsize=(21,28))

    # Plot boxes - Area thresholded
    plt.subplot(3,2,1)
    plt.imshow(img, cmap='gray', vmin=0, vmax=255)
    
    plt.subplot(3,2,2)
    plt.imshow(img_canny, cmap='gray')
    plt.colorbar()
    
    plt.subplot(3,2,3)
    plt.imshow(img_dil, cmap='gray', vmin=0, vmax=255)
    
    plt.subplot(3,2,4)
    plt.imshow(mask, cmap='gray', vmin=0, vmax=255)
    ax = plt.gca()
    
    # Boxes in evaluation
    plt.subplot(3,2,5)
    plt.imshow(mask, cmap='gray', vmin=0, vmax=255)
    ax = plt.gca()

    for i, box in enumerate(intersection_boxes):
        x,y,w,h = box # upper left
        rect = patches.Rectangle((x,y), w,h, edgecolor='y', facecolor='y', fill=True, linewidth=2, alpha=0.25)
        ax.add_patch(rect)     
    
    for i, box in enumerate(pred_boxes):
        x,y,w,h = box # upper left
        rect = patches.Rectangle((x,y), w,h, edgecolor='r', fill=False, linewidth=4)
        ax.add_patch(rect)
        
    for i, box in enumerate(true_boxes):
        x,y,w,h = box # upper left
        rect = patches.Rectangle((x,y), w,h, edgecolor='g', fill=False, linewidth=4)
        ax.add_patch(rect)
        
    
    # All boxes
    plt.subplot(3,2,6)
    plt.imshow(mask, cmap='gray', vmin=0, vmax=255)
    ax = plt.gca()
    
    for i, box in enumerate(pred_boxes):
        x,y,w,h = box # upper left
        rect = patches.Rectangle((x,y), w,h, edgecolor='r', fill=False, linewidth=4)
        ax.add_patch(rect)
    
    for i, box in enumerate(pre_land_boxes):
        x,y,w,h = box # upper left
        rect = patches.Rectangle((x,y), w,h, edgecolor='r', fill=False, linewidth=4)
        ax.add_patch(rect)
        
    for i, box in enumerate(true_boxes):
        x,y,w,h = box # upper left
        rect = patches.Rectangle((x,y), w,h, edgecolor='g', fill=False, linewidth=4)
        ax.add_patch(rect)
    
    
        
     
    fig.suptitle(title + ' | Found: {}, Truth: {}'.format(len(pred_boxes), len(true_boxes)), fontsize=18)
    plt.show()